## Day 2: Native Bayes
Step 1: 准备数据，从文本中构建词向量

In [ ]:
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 is abusive, 0 not
    return postingList,classVec

def createVocabList(dataset):
    # 创建一个空的集合 http://www.iplaypy.com/jichu/set.html
    vocabSet = set([])
    for document in dataset:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

# 词集模型，单词出现的次数不作为特征，只考察是否出现
def setOfWords2Vec(vocabList, inputSet):
    # 创建一个其中所有元素都是0的向量
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print ("the word %s is not in my vocabulary!" % word)
    return returnVec

# 词袋模型，一个单词的出现的次数被考虑进来
def bagOfWords2Vec(vocabList, inputSet):
    # 创建一个其中所有元素都是0的向量
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
        else:
            print ("the word %s is not in my vocabulary!" % word)
    return returnVec

# func test
listOposts, listClasses = loadDataSet()
myVocabList = createVocabList(listOposts)
returnVec = setOfWords2Vec(myVocabList, listOposts[0])

Step 2: 训练算法，从词向量计算概率

Step 3: 测试算法，根据现实情况修改分类器（注释掉的即使修改过的代码）

In [26]:
from numpy import * # 调用NumPy数组
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
#     p0Num = zeros(numWords)
#     p1Num = zeros(numWords)
#     p0Denom = 0.0
#     p1Denom = 0.0
    # 此时修改是因为需要算，概率乘积，其中有一个为0，结果便是0，所以，将所有单词都初始化为1
    p0Num = ones(numWords)
    p1Num = ones(numWords)
    p0Denom = 2.0
    p1Denom = 2.0
    # 用NumPy数组来快速计算这些值
    for i in range(numTrainDocs):
        if trainCategory[i] == 0:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
        else:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
#     p1Vect = p1Num/p1Denom
#     p0Vect = p0Num/p0Denom
    # 下溢出，很多很小的数，相乘会下溢出，所以，可以用取自然对数后，转变为加法
    p1Vect = log(p1Num/p1Denom)
    p0Vect = log(p0Num/p0Denom)
    return p0Vect,p1Vect, pAbusive

# func test
trainMat = []
for i in range(len(listClasses)):
    trainMat.append(setOfWords2Vec(myVocabList, listOposts[i]))
    
p0V,p1V, pAb = trainNB0(trainMat, listClasses)

# 朴素贝叶斯分类函数
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify*p1Vec) + log(pClass1)
    p0 = sum(vec2Classify*p0Vec) + log(1-pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

# func test
testEntry = ['stupid', 'garbage']
thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
classifyNB(thisDoc, p0V,p1V, pAb)

1